In [32]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt


In [33]:
data =  pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
data.head()


,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
data = np.array(data)
m, n = data.shape
np.random.shuffle(data) # shuffle before splitting into dev and training sets


data_dev = data[0:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = X_dev / 255.

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.
_,m_train = X_train.shape

In [35]:
def init_params():
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2

def ReLU(Z):
    return np.maximum(Z, 0)

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A
    
def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def ReLU_deriv(Z):
    return Z > 0 # activation function

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2    
    return W1, b1, W2, b2

In [36]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2

In [37]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.10, 500)

Iteration:  0
[6 6 5 ... 6 3 6] [6 0 4 ... 9 7 3]
0.103
Iteration:  10
[5 2 5 ... 2 5 2] [6 0 4 ... 9 7 3]
0.15129268292682926
Iteration:  20
[5 5 5 ... 1 5 3] [6 0 4 ... 9 7 3]
0.2258048780487805
Iteration:  30
[5 5 5 ... 3 5 3] [6 0 4 ... 9 7 3]
0.2627317073170732
Iteration:  40
[5 0 5 ... 3 5 3] [6 0 4 ... 9 7 3]
0.2856341463414634
Iteration:  50
[5 0 5 ... 1 5 3] [6 0 4 ... 9 7 3]
0.33002439024390245
Iteration:  60
[5 0 5 ... 1 0 3] [6 0 4 ... 9 7 3]
0.3758048780487805
Iteration:  70
[6 0 5 ... 1 7 3] [6 0 4 ... 9 7 3]
0.40841463414634144
Iteration:  80
[6 0 5 ... 1 7 3] [6 0 4 ... 9 7 3]
0.43158536585365853
Iteration:  90
[6 0 5 ... 1 7 3] [6 0 4 ... 9 7 3]
0.4491951219512195
Iteration:  100
[6 0 5 ... 1 7 3] [6 0 4 ... 9 7 3]
0.4650487804878049
Iteration:  110
[6 0 4 ... 9 7 3] [6 0 4 ... 9 7 3]
0.48860975609756097
Iteration:  120
[6 0 4 ... 9 7 3] [6 0 4 ... 9 7 3]
0.5179268292682927
Iteration:  130
[6 0 4 ... 9 7 3] [6 0 4 ... 9 7 3]
0.5422439024390244
Iteration:  140
[6 0 4 ..

In [38]:
test_prediction(102,W1,b1,W2,b2)

NameError: name 'test_prediction' is not defined